Load Video

1.   Load from Google Drive in Colab
2.   Make directory



In [9]:
# Google Colab import files in google drive
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()

from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
!mkdir -p drive
!google-drive-ocamlfuse drive
!cd drive; ls; 

gpg: keybox '/tmp/tmpmz0cj106/pubring.gpg' created
gpg: /tmp/tmpmz0cj106/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
··········
fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option
4월 27일 JUNO 한강 스터디.zip  Cropped_imgs    웹크롤러 추가 기능 제안서.ods
ColabNotebooks		       FaceClustering  학점시발.ods
Colab Notebooks		       new_file.txt    Windows10


In [2]:
!pip install tqdm
from tqdm import tqdm
import cv2 as cv
import numpy as np
import os
import datetime

In [0]:
VIDEO_DIR = 'drive/FaceClustering/'
VIDEO_FILE_NAME = 'WINNER_EVERYDAY.mp4'
PROGRAM_START_TIME = datetime.datetime.now().strftime('%Y%m%d_%H-%M-%S')
SAVE_CROPPED_FACE_DIR = 'drive/Cropped_imgs/' + PROGRAM_START_TIME + '-' + VIDEO_FILE_NAME.split('.')[0]
CASCADE_DIR = 'drive/FaceClustering/lbpcascade_frontalface_improved.xml'

In [0]:
def make_directory(_directory_name):
    if not os.path.exists(_directory_name):
        os.makedirs(_directory_name)

In [0]:
def get_area_of_frame_face_recognition(img, face_cascade):
    grayed_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    # def detectMultiScale(self, image, scaleFactor=None, minNeighbors=None, flags=None, minSize=None, maxSize=None)
    face_area = face_cascade.detectMultiScale(image=grayed_img,scaleFactor=1.3,minNeighbors=5)
    return face_area

In [0]:
def get_frame(video_capture, play_speed=1):
    curr_frame = video_capture.get(cv.CAP_PROP_POS_FRAMES)
    video_capture.set(cv.CAP_PROP_POS_FRAMES, curr_frame + play_speed)
    ret, img = video_capture.read()
    fps = video_capture.get(cv.CAP_PROP_FPS)
    return img, (curr_frame + play_speed) / fps

In [0]:
def save_cropped_img(img, faces, sec):
    if len(faces) != 0:
        i = 0
        for (x, y, w, h) in faces:
#             print(SAVE_CROPPED_FACE_DIR + "/" + str(sec).replace('.','_') + "_" + str(i) + ".jpg")
#             cv.imshow('hello', img[y:y + h, x: x + w])
            cv.imwrite(SAVE_CROPPED_FACE_DIR + "/" + str(sec).replace('.','_') + "_" + str(i) + ".jpg", img[y:y + h, x: x + w])
            i += 1
    if cv.waitKey(1) & 0xFF == ord('q'):
        return False
    else:
        return True


In [0]:
make_directory(SAVE_CROPPED_FACE_DIR)
# load video
m_video_capture = cv.VideoCapture(VIDEO_DIR + VIDEO_FILE_NAME)
m_face_cascade = cv.CascadeClassifier(CASCADE_DIR)
TOTAL_FRAME = m_video_capture.get(cv.CAP_PROP_FRAME_COUNT)

In [9]:
FRAME_STEP = 2
for frame in tqdm(range(0, int(TOTAL_FRAME), FRAME_STEP)):
    img, sec = get_frame(video_capture=m_video_capture, play_speed=FRAME_STEP)
    faces_area = get_area_of_frame_face_recognition(img=img, face_cascade=m_face_cascade)
    save_cropped_img(img, faces_area, sec)

 67%|██████▋   | 1712/2568 [11:52<05:56,  2.40it/s]

error: ignored